<a href="https://colab.research.google.com/github/jhk990602/datapractice/blob/main/%EC%A0%84%EC%B2%98%EB%A6%AC%EA%B3%BC%EC%A0%95(%EB%85%B8%EB%8B%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    classification_report,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

pd.options.display.max_columns = None

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/lg/train.csv')

In [ ]:
data.head()

In [ ]:
"""

# 1. 불필요한 열 제거
unnamed_columns = [col for col in data.columns if 'Unnamed' in col]
data = data.drop(columns=unnamed_columns)

# 2. 타겟 변수 'is_converted' 처리
data['is_converted'] = data['is_converted'].map({True: 1, False: 0})

# 3. 결측치 처리
# 결측치가 많은 열 제거 (예: 결측치가 전체의 50% 이상인 열)
half_len = len(data) / 2
data = data.dropna(thresh=half_len, axis=1)

# 수치형 데이터 열의 결측치는 중앙값으로 채우기
for column in data.select_dtypes(include=['float64', 'int64']).columns:
    data[column].fillna(data[column].median(), inplace=True)

# 범주형 데이터 열의 결측치는 최빈값으로 채우기
for column in data.select_dtypes(include=['object']).columns:
    data[column].fillna(data[column].mode()[0], inplace=True)

# 4. 범주형 데이터 인코딩
for column in data.select_dtypes(include=['object']).columns:
    data[column] = pd.Categorical(data[column]).codes

# 데이터 전처리 완료 후 최종 데이터셋 크기 확인
print("최종 데이터셋 크기:", data.shape)

"""

In [ ]:
# / 슬래시 2개 뒤에 보통 국가이름, 그냥 국가만 적혀있거나 메일형태 or . 으로 나뉜경우도 있
#df['연도']=list(map(lambda x : int(x.split(".")[0].split()[0].split("/")[0]), df["기간"]))
data['clean_country']=data['customer_country'].apply(lambda x : str(x).split('/')[-1])

In [ ]:
#@있는거빼기
data.loc[data['clean_country'].str.contains('@', na=False), 'clean_country'] = np.nan

In [ ]:
India_list = ['Anand Vihar Delhi', 'Botswana', 'Chennai', 'Pune', 'Telangana', 'bangalore', 'bhilwara', 'gujarat', 'gurgaon', 'hyderabad', 'indore', 'kerela', 'mumbai', 'odisha', 'uttar pradesh']
Spain_list=['ALICANTE','CACERES','GRAN CANARIAS PLAYA DEL INGLES','MADRID','VALENCIA','spain']
Brazil_list=['Brazil','Aparecida','Belo Horizonte','Capão da canoa','Centro de Produção Audiovisual - Sesc São Paulo','Cuiabá','Dourados','Foz de Iguaçu - PR Avenida Tancredo Neves 6731 Jardim Itaipu','João Pessoa','Manaus','Recife','Rio de Janeiro','Sao Paulo','São Paulo']
Colombia_list=['Barranquilla','Bucaramanga','BOGOTA','Cartagena','colombia']
Vietnam_list=['Ha Noi','vietnam']
Turkey_list=['Türkiye','Turkey']
UAE_list=['UAE Dubai']#U.A.E로 넣어야함
Greece_list=['Greece','Θέση Πέτσα Βακαλοπούλου ΒΙΟΠΑ Παλλήνης  153 51']
Italy_list=['italy','90044 Carini']
Netherlands_list=['Netherlands']
CotedIvoire_list=['Ivory Coast']#Cote d'Ivoire 로 등록하자
Saint_Kitts_list=['St Kitts'] #Saint Kitts and Nevis로 등록하자
Puerto_Rico_list=['Puerto Rico'] #Puerto Rico로 등록하자
USA_list=['US','United States','Jacksonville Florida','New Hampshire','Nevada','Ohio','Zip 98433']#United States로 등록하자
Peru_list=['PerU']

country_lists = [India_list,Spain_list, Brazil_list, Colombia_list, Vietnam_list, Turkey_list, UAE_list, Greece_list,Italy_list,Netherlands_list,CotedIvoire_list,Saint_Kitts_list,Puerto_Rico_list,USA_list,Peru_list]
country_names = ['India','Spain', 'Brazil', 'Colombia', 'Vietnam', 'Turkey', 'U.A.E', 'Greece','Italy','Netherlands',"Cote d'Ivoire",'Saint Kitts and Nevis','Puerto Rico','United States','Peru']

for i in range(len(country_lists)):
    for city in country_lists[i]:
        data.loc[data['clean_country'].str.contains(city, case=False, na=False), 'clean_country'] = country_names[i]

In [ ]:
#숫자들어간것 다 미국으로 바꾸기
data.loc[data['clean_country'].str.contains('\d', regex=True, na=False), 'clean_country'] = 'United States'

In [ ]:
#맨앞글자가 공백이면 공백없에기?
data['clean_country'] = data['clean_country'].str.lstrip()

In [ ]:
set(data['clean_country'])
#A,BR,CA,ND,NY,RJ,'country','lucknow','nan'만처리하면될듯?

In [ ]:
print(data['clean_country'].value_counts())

In [ ]:
#이메일싹다드랍, 띄어쓰기 자르고, 대소문자 나누고, 도시이름은 그냥 포함해서 날리는것을 디폴트로 생각
#그냥 country 드랍
#인도
#data['clean_country'] = np.where(data['clean_country'].str.contains('Delhi','Chennai','Pune','Telangana','bangalore','bhilwara','gujarat','gurgaon','hyderabad','indore','kerela','mumbai','odisha','uttar pradesh'), 'India', data['clean_country'])

india=[' Anand Vihar Delhi',' Botswana',' Chennai',' Pune',' Telangana',' bangalore',' bhilwara',' gujarat',' gurgaon',' hyderabad',' indore',' kerela',' mumbai',' odisha',' uttar pradesh','lucknow','mumbai']


#숫자가 들어가지만 미국애매한거 '1605 Ave. Ponce de León, Suite 400 San Juan, 00909, Puerto Rico',
Spain_list=['ALICANTE','CACERES','GRAN CANARIAS PLAYA DEL INGLES','MADRID','VALENCIA','spain']
Brazil_list=['Brazil','Aparecida','Belo Horizonte','Capão da canoa','Centro de Produção Audiovisual - Sesc São Paulo','Cuiabá','Dourados','Foz de Iguaçu - PR Avenida Tancredo Neves 6731 Jardim Itaipu','João Pessoa','Manaus','Recife','Rio de Janeiro','Sao Paulo','São Paulo']
Colombia_list=['Barranquilla','Bucaramanga','BOGOTA','Cartagena','colombia']
Vietnam_list=['Ha Noi','vietnam']
Turkey_list=['Türkiye','Turkey']
UAE_list=['UAE Dubai']#U.A.E로 넣어야함
Greece_list=['Greece','Θέση Πέτσα Βακαλοπούλου ΒΙΟΠΑ Παλλήνης  153 51']
Italy_list=['italy','90044 Carini']
Netherlands_list=['Netherlands']
CotedIvoire_list=['Ivory Coast']#Cote d'Ivoire 로 등록하자
Saint_Kitts_list=['St Kitts'] #Saint Kitts and Nevis_list로 등록하자
Puerto_Rico_list=['Puerto Rico'] #Puerto Rico로 등록하자
USA_list=['Jacksonville Florida','New Hampshire','Nevada','Ohio','Zip 98433']

#코트디부아르가 Cote d'Ivoire랑 Ivory Coast 2개인것 인
#Virgin Islands 포함은 Virgin Islands로 통일? 숫자들어있어서 미리처리필
#Italy 포함은 italy로 바꾸는거 까먹x, italy=['90044 Carini']
#Netherlands포함은 Netherlands

#Saint Kitts and Nevis=St Kitts
#US Virgin Islands=그냥 Virgin Islands도 있어서 미국하기전에 따로 빼기
#하다보니까 국가가 너무 다양해서 쓸모없을것같다는 생각
#Puerto Rico 종류 2개였던듯 다시보기

In [ ]:
#미국(RJ 드랍할지) #contain안쓸거
"""
ca_addresses = ['CA 94555 United States', 'CA 92657 United States', ...]
df_train['customer_country'] = df_train['customer_country'].replace(ca_addresses, 'United States')
"""
data['clean_country'] = np.where(data['clean_country'].str.contains('CA','FL','TX','CO 80127','OK','IL 6','IN 46601','USA','MA 0','MO 6','1 Hoag Dr'), 'United States', data['clean_country'])
data['clean_country'] = np.where(data['clean_country'].str.contains('^ND','^NJ','^NV','^US','^UT''^VA','^VT','^Warren'), 'United States', data['clean_country'])

#숫자들어간거 90프로 미국, italy나 몇개 거르고 정수는 싹다 미국으로 돌리기, 터키 종류가 많음. 다 거르고 상관계수 확인필요할듯.

In [ ]:
#결측치를 날려버리면 customer_idx가 연관도가 가장높게 0.6 안날리면 com_reg가 가장 높게 0.6정도로 잡힘
#customer_type은 애초에 결측치너무많음 + 채우기애매 + 연관도가 너무낮아서 drop이 나을듯

In [ ]:
#필요없어보이는것 > customer_type,id it idit,
#inquiry 문의유형은 한번 시간될때 건들여보는것도

In [ ]:
#product_category 위에 숫자는 모니터를 뜻함. 다 건들필요있을 수도. nan이 19374개
#value count를 보면 Interactive Digital Board는 모니터, vrf는 에어컨쪽 multisplit은 에어컨 각방조절? video wall은 큰광고판화면같
#lg signage는 애매, single-split도 에어컨 rac도 에어컨쪽 chiller 에어컨부품? 냉매 one quick은 모니터?

In [ ]:
#data['product_subcategory']와 data['product_category'] 를 비교해서 nan을 채울까
#data['product_modelname']랑도 연관 지으려면 지을수있

In [ ]:
#null값이 있지만 국가로 나눌지, response_corporate로 법인으로 채워넣을지
#businerss_area를 쓸필요가있나 엮으려면 엮는데
